In [1]:
import joblib

import numpy as np
import networkx as nx

from src.CBN import CausalBayesianNetwork as CBN
import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops
import params

### Define low-level DCM "LL" and high-level DCM "HL"

In [2]:
experiment = 'synth1'

In [3]:
S = 'Smoking'
T = 'Tar'
C = 'Cancer'

S_ = 'Smoking_'
C_ = 'Cancer_'

In [4]:
ll_endogenous_coeff_dict = {(S, T): 0.3, (T, C): 0.2}
ll_causal_graph          = CBN(list(ll_endogenous_coeff_dict.keys()))
#nx.draw(nx.DiGraph(ll_causal_graph.edges()),with_labels=True)

hl_endogenous_coeff_dict = {(S_, C_): 0.0}
hl_causal_graph          = CBN(list(hl_endogenous_coeff_dict.keys()))
# #nx.draw(nx.DiGraph(hl_causal_graph.edges()),with_labels=True)

In [5]:
# Define the number of samples from the low-level environment.
num_llsamples   = params.n_samples[experiment][0]

In [63]:
torch.randn(3)

tensor([-0.7679,  0.4804, -0.3191])

### Construct the empirical nominal distribution/ environment

In [6]:
ll_mu_hat    = np.array([0, 0, 0])  
ll_Sigma_hat = np.diag([1, 1, 1]) 

### Define the sets of relevant interventions and the (total) surjective and order-preserving function $ω:I^{L} \mapsto I^{H}$

In [7]:
iota0 = None
iota1 = ops.Intervention({S:0})
iota2 = ops.Intervention({S:0, T:1})
iota3 = ops.Intervention({S:1})
iota4 = ops.Intervention({S:1, T:0})
iota5 = ops.Intervention({S:1, T:1})

eta0 = None
eta1 = ops.Intervention({S_:0})
eta2 = ops.Intervention({S_:1})

omega = {   
            iota0: eta0,
            iota1: eta1,
            iota2: eta1,
            iota3: eta2,
            iota4: eta2,
            iota5: eta2
        }

Ill_relevant = list(set(omega.keys()))
Ihl_relevant = list(set(omega.values()))

### Sampling and Pair construction

In [92]:
T = np.array([[1, 2, 1], [0, 1, 0]])

In [96]:
Dll_samples, Dll_noise, Dhl_samples = {}, {}, {}
for iota in Ill_relevant:
    llcm              = lanm.LinearAddSCM(ll_causal_graph, ll_endogenous_coeff_dict, iota)
    
    #Different Dll_noise for each iota
    lenv_iota         = mut.sample_distros_Gelbrich([(ll_mu_hat, ll_Sigma_hat)])[0] 
    Dll_noise[iota]   = lenv_iota.sample(5)[0]
    Dll_samples[iota] = llcm.sample_settings(Dll_noise[iota])
    Dhl_samples[omega[iota]] = Dll_samples[iota] @ T.T

In [98]:
for iota in Ill_relevant:
    if iota is not None:
        print(iota.vv())
    else:
        print(iota)

None
{'Smoking': 0, 'Tar': 1}
{'Smoking': 1, 'Tar': 0}
{'Smoking': 1}
{'Smoking': 0}
{'Smoking': 1, 'Tar': 1}


In [100]:
cur_list = []
i = 0
for iota in Ill_relevant:
    if iota is not None:
        print(iota.vv(),'--',omega[iota].vv())
    else:
        print(iota,'--', omega[iota])
    if omega[iota] not in cur_list:
        Dhl_samples[Ihl_relevant[i]] = Dll_samples[iota]@T.T
        cur_list.append(Ihl_relevant[i])
        i += 1

None -- None
{'Smoking': 0, 'Tar': 1} -- {'Smoking_': 0}
{'Smoking': 1, 'Tar': 0} -- {'Smoking_': 1}
{'Smoking': 1} -- {'Smoking_': 1}
{'Smoking': 0} -- {'Smoking_': 0}
{'Smoking': 1, 'Tar': 1} -- {'Smoking_': 1}


In [103]:
for iota in Ill_relevant:
    print(Dhl_samples[iota],'---',Dll_samples[iota]@T.T)

[[ 1.61672621  0.7208437 ]
 [ 1.2531752  -0.52818454]
 [ 0.66325701  0.18840785]
 [ 1.06495801 -0.39740576]
 [ 0.32929939 -1.29155702]] --- [[ 1.61672621  0.7208437 ]
 [ 1.2531752  -0.52818454]
 [ 0.66325701  0.18840785]
 [ 1.06495801 -0.39740576]
 [ 0.32929939 -1.29155702]]


KeyError: <operations.Intervention object at 0x195bca9c0>

In [11]:
Ds = {}
for iota in Ill_relevant:
    Ds[iota] = (Dll_samples[iota], Dhl_samples[omega[iota]])

In [12]:
hl_coeffs = mut.get_coefficients(Dhl_samples[None], hl_causal_graph) 

In [13]:
Dhl_noise, hl_mu_hat, hl_Sigma_hat = mut.lan_abduction(Dhl_samples[None], hl_causal_graph, hl_coeffs)

# hl_moments     = [(hl_mu_hat, hl_Sigma_hat)]
# henv           = mut.sample_distros_Gelbrich(hl_moments)[0] 

In [14]:
joblib.dump((ll_causal_graph, Ill_relevant), f"data/{experiment}/LL.pkl")
joblib.dump(ll_endogenous_coeff_dict, f"data/{experiment}/ll_coeffs.pkl")

joblib.dump((hl_causal_graph, Ihl_relevant), f"data/{experiment}/HL.pkl")
joblib.dump(hl_coeffs, f"data/{experiment}/hl_coeffs.pkl")

joblib.dump(Ds, f"data/{experiment}/Ds.pkl")

joblib.dump(T, f"data/{experiment}/Tau.pkl")
joblib.dump(omega, f"data/{experiment}/omega.pkl")

joblib.dump((Dll_noise[None], ll_mu_hat, ll_Sigma_hat), f"data/{experiment}/exogenous_LL.pkl")
joblib.dump((Dhl_noise, hl_mu_hat, hl_Sigma_hat), f"data/{experiment}/exogenous_HL.pkl")

['data/synth1/exogenous_HL.pkl']